In [6]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

import ast

In [9]:
idf = pd.read_csv('../TRAIN_SAL.csv')
idf.head(100)

/tmp/ipykernel_3000/1125481573.py:1: DtypeWarning: Columns (1,16,24,69) have mixed types. Specify dtype option on import or set low_memory=False.
  idf = pd.read_csv('../TRAIN_SAL.csv')


,id,academic_degree,accommodation_capability,accommodation_type,additional_premium,additional_requirements,bonus_type,measure_type,busy_type,career_perspective,change_time,code_external_system,code_profession,code_professional_sphere,company_code,contact_person,contact_source,data_ids,date_create,date_modify,deleted,education,education_speciality,foreign_workers_capability,metro_ids,is_mobility_program,is_moderated,is_uzbekistan_recruitment,is_quoted,need_medcard,oknpo_code,okso_code,original_source_type,other_vacancy_benefit,position_requirements,position_responsibilities,publication_period,published_date,regionName,regionNameTerm,company_business_size,required_certificates,required_drive_license,required_experience,retraining_capability,retraining_condition,retraining_grant,retraining_grant_value,salary,salary_min,salary_max,schedule_type,social_protected_ids,source_type,state_region_code,status,transport_compensation,vacancy_address_additional_info,vacancy_address,vacancy_address_code,vacancy_address_house,vacancy_address_latitude,vacancy_address_longitude,vacancy_benefit_ids,vacancy_name,visibility,work_places,professionalSphereName,federalDistrictCode,industryBranchName,contactList,company_name,full_company_name,company_inn,company,languageKnowledge,hardSkills,softSkills
0,4e2f52d2-fd76-11e4-8a5b-3bdbd1a6e39d,NaN,False,NaN,NaN,"<p>Доплаты молодым специалистам, возможность с...",NaN,NaN,Полная занятость,False,2024-07-03T10:38:20+0300,CZN,240387.0,Medicine,1023501241390,Наталья Александровна,COMPANY,"[""a0b42430-422d-11e5-98d7-3bdbd1a6e39d"",""a0b42...",2015-06-27T09:27:34+0300,2024-07-03T13:16:03+0300,False,Среднее профессиональное,Сестринское дело,NaN,NaN,False,True,False,False,NaN,NaN,NaN,EMPLOYMENT_SERVICE,"<p>Социальный пакет, ч<span style=""font-family...",<p>Наличие сертификата или аккредитации по спе...,<p>-осуществление сестринского процесса при ух...,0.0,2024-07-03T13:16:03+0300,Вологодская область,NaN,MIDDLE,Наличие сертификата или аккредитации по специа...,[],0.0,False,NaN,нет стипендии,NaN,33500,33500.0,50000.0,Сменный график,NaN,Работодатель,3.500000e+12,ACCEPTED,NaN,151 а,"Вологодская область, Город Череповец, Металлур...",3.500000e+16,55,59.132305,37.882854,NaN,Медицинская сестра палатная (постовая),VISIBLE_TO_ALL,1,Здравоохранение и социальное обеспечение,2.0,NaN,NaN,NaN,БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ ВОЛОГОДСК...,3.528053e+09,"{""companycode"":""1023501241390"",""email"":""admin@...",[],[],[]
1,fafe2e23-0e2c-11e5-9cc4-3bdbd1a6e39d,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,2024-03-12T15:55:59+0300,CZN,240387.0,Medicine,1104714000115,Алевтина Загороднюк,CZN,"[""38b1b0c0-36c3-11e5-a8c4-239645b044d5"",""cd1d7...",2015-06-27T09:28:58+0300,2024-06-24T16:38:03+0300,False,Среднее профессиональное,NaN,NaN,NaN,NaN,True,False,False,NaN,NaN,NaN,EMPLOYMENT_SERVICE,<ul> <li>профессиональное развитие в стабильно...,<ul> <li>среднее специальное медицинское образ...,<ul> <li>подготовка кабинета к приему пациенто...,0.0,2024-06-24T16:38:03+0300,Ленинградская область,NaN,SMALL,Сестринское дело,[],0.0,False,NaN,нет стипендии,NaN,60000,60000.0,80000.0,Сменный график,NaN,Работодатель,4.700000e+12,ACCEPTED,NaN,"188544, г Сосновый Бор, ул Молодежная, д. 9, к...","Ленинградская область, Город Сосновый Бор, Мол...",4.700000e+16,NaN,59.910871,29.092246,NaN,Медицинская сестра,VISIBLE_TO_ALL,1,Здравоохранение и социальное обеспечение,2.0,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МЕДИ...",4.714024e+09,"{""companycode"":""1104714000115"",""email"":""llm289...",[],[],[]
2,7eb9edc1-fd76-11e4-8a5b-3bdbd1a6e39d,NaN,False,NaN,NaN,"<p>&nbsp;</p> <p>Фельдшер отделения СМП, Ковжи...",NaN,NaN,Полная занятость,False,2024-04-11T11:52:05+0300,CZN,273287.0,Medicine,1023502092360,Постнова Антонина Сергеевна,COMPANY,"[""c76b1c40-be43-11e5-bbaf-3bdbd1a6e39d"",""73677...",2015-06-27T09:29:16+0300,2024-06-11T08:43:02+0300,False,Среднее профессиональное,NaN,NaN,NaN,False,True,False,False,True,NaN,NaN,EMPLOYMENT_SERVICE,<p>Зд

In [10]:
df = idf


In [11]:
print(len(df['code_profession'].unique()))
print(len(df['code_professional_sphere'].unique()))

4424
35


In [12]:
len(df)

631117

In [13]:
###preprocess data
def tokenize(vacancy_name):
    # Tokenize the vacancy name using regular expressions
    return set(re.split(r'[^\w]+', vacancy_name.lower()))

def compute_token_intersections(tokens1, tokens2):
    # Compute the intersection score between two sets of tokens
    return len(tokens1 & tokens2)

tokens_dict_profession = {}
code_profession_dict = {}


def impute_values(df, is_train, train_df=None):
    global tokens_dict_profession, code_profession_dict
    if is_train:
        # Handling 'code_profession'
        non_nan_train = train_df[train_df['code_profession'].notna()]
        vacancy_to_code_profession = non_nan_train.set_index('vacancy_name')['code_profession'].to_dict()
        df['code_profession'] = df['vacancy_name'].map(vacancy_to_code_profession).fillna(df['code_profession'])

        for index, row in df[df['code_profession'].notna()].iterrows():
            tokens_dict_profession[index] = tokenize(row['vacancy_name'])
            code_profession_dict[index] = row['code_profession']

        for index, row in df[df['code_profession'].isna()].iterrows():
            current_tokens = tokenize(row['vacancy_name'])
            max_intersection = 0
            best_match = None

            for other_index, other_tokens in tokens_dict_profession.items():
                intersection_score = compute_token_intersections(current_tokens, other_tokens)

                if intersection_score > max_intersection:
                    max_intersection = intersection_score
                    best_match = code_profession_dict[other_index]
                if intersection_score == 2 or intersection_score == len(current_tokens):
                    break

            df.at[index, 'code_profession'] = best_match

        df['code_profession'].fillna(df['code_profession'].median(), inplace=True)

        # Train set imputation for 'code_professional_sphere'
        non_nan_train = train_df[train_df['code_professional_sphere'].notna()]
        vacancy_to_code_professional_sphere = non_nan_train.set_index('vacancy_name')['code_professional_sphere'].to_dict()

        df['code_professional_sphere'] = df['vacancy_name'].map(vacancy_to_code_professional_sphere).fillna(df['code_professional_sphere'])

        tokens_dict_professional_sphere = {}
        code_professional_sphere_dict = {}

        for index, row in df[df['code_professional_sphere'].notna()].iterrows():
            tokens_dict_professional_sphere[index] = tokenize(row['vacancy_name'])
            code_professional_sphere_dict[index] = row['code_professional_sphere']

        for index, row in df[df['code_professional_sphere'].isna()].iterrows():
            current_tokens = tokenize(row['vacancy_name'])
            max_intersection = 0
            best_match = None

            for other_index, other_tokens in tokens_dict_professional_sphere.items():
                intersection_score = compute_token_intersections(current_tokens, other_tokens)

                if intersection_score > max_intersection:
                    max_intersection = intersection_score
                    best_match = code_professional_sphere_dict[other_index]
                if intersection_score == 2 or intersection_score == len(current_tokens):
                    break

            df.at[index, 'code_professional_sphere'] = best_match

        return df

    else:
        # Test set imputation based on train_df
        if train_df is None:
            raise ValueError("train_df must be provided for test set imputation")

        # Handling 'code_profession'
        non_nan_train = train_df[train_df['code_profession'].notna()]
        vacancy_to_code_profession = non_nan_train.set_index('vacancy_name')['code_profession'].to_dict()

        df['code_profession'] = df['vacancy_name'].map(vacancy_to_code_profession)
        #print(df['code_profession'])
        #df['code_profession'].fillna(df['code_profession'].apply(
        #    lambda x: find_best_match(x, tokens_dict_profession, code_profession_dict)), inplace=True)
        # Handling 'code_professional_sphere'
        non_nan_train = train_df[train_df['code_professional_sphere'].notna()]
        vacancy_to_code_professional_sphere = non_nan_train.set_index('vacancy_name')['code_professional_sphere'].to_dict()

        df['code_professional_sphere'] = df['vacancy_name'].map(vacancy_to_code_professional_sphere)
        #df['code_professional_sphere'].fillna(df['code_professional_sphere'].apply(
        #    lambda x: find_best_match(x, tokens_dict_professional_sphere, code_professional_sphere_dict)), inplace=True)

        return df

def find_best_match(vacancy_name, tokens_dict, code_dict):
    current_tokens = tokenize(vacancy_name)
    max_intersection = 0
    best_match = None

    for other_index, other_tokens in tokens_dict.items():
        intersection_score = compute_token_intersections(current_tokens, other_tokens)

        if intersection_score > max_intersection:
            max_intersection = intersection_score
            best_match = code_dict[other_index]
        if intersection_score == 2 or intersection_score == len(current_tokens):
            break
    return best_match

categories_which_need_medcard = set()

def needs_medcard(category):
    return category in categories_which_need_medcard

def hash_col(df, col, N):
    cols = [col + "_" + str(i) for i in range(N)]

    # Preallocate a zero matrix
    result = np.zeros((df.shape[0], N), dtype=int)

    # Apply hash function and set the corresponding column to 1
    for idx, val in enumerate(df[col]):
        result[idx, hash(val) % N] = 1

    # Create DataFrame from the result matrix
    df_hash = pd.DataFrame(result, columns=cols)

    # Concatenate the new columns to the original DataFrame and drop the old column
    return pd.concat([df.drop(col, axis=1), df_hash], axis=1)


premium_scaler = None
experience_scaler = None
salary_scaler = None
places_scaler = None
encoders = {
    'academic_degree': OneHotEncoder(sparse=False, handle_unknown='ignore'),
    'accommodation_type': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'bonus_type': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'measure_type': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'busy_type': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'education': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'original_source_type': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'company_business_size': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'schedule_type': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'social_protected_ids': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'source_type': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'state_region_code': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'status': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'transport_compensation': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'vacancy_benefit_ids': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'professionalSphereName': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'federalDistrictCode': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'code_professional_sphere': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'required_drive_license': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    'languageKnowledge': OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    
}


def encode_column_train(df, column_name, prefix):
    
    encoders[column_name].fit(df[[column_name]])
    encoded = encoders[column_name].transform(df[[column_name]])
    categories = encoders[column_name].categories_[0]  # Assuming one feature
    feature_names = [f"{prefix}_{cat}" for cat in categories]
    encoded_df = pd.DataFrame(encoded, columns=feature_names)
    #encoded_df = pd.DataFrame(encoded, columns=encoders[column_name].get_feature_names_out([prefix]))
    
    # Ensure the encoded DataFrame has the same index as the original
    encoded_df.index = df.index
    
    return encoded_df


def encode_column(df, column_name, prefix):
    encoded = encoders[column_name].transform(df[[column_name]])
    
    encoded_df = pd.DataFrame(encoded, columns=encoders[column_name].get_feature_names_out([prefix]))
    
    # Ensure the encoded DataFrame has the same index as the original
    encoded_df.index = df.index
    
    return encoded_df


def explode_and_encode_train(df, column_name, prefix, value=None):
    df[column_name] = df[column_name].apply(ast.literal_eval)
    if value is not None:
        exploded = df[column_name].apply(lambda x: [d[value] for d in x]).explode()
    else:
        exploded = df[column_name].explode()
    encoded = encoders[column_name].fit_transform(exploded.values.reshape(-1, 1))
    encoded_df = pd.DataFrame(encoded, columns=encoders[column_name].get_feature_names_out([prefix]))
    return encoded_df.groupby(level=0).sum()


def explode_and_encode(df, column_name, prefix, value=None):
    df[column_name] = df[column_name].apply(ast.literal_eval)
    if value is not None:
        exploded = df[column_name].apply(lambda x: [d[value] for d in x]).explode()
    else:
        exploded = df[column_name].explode()
    encoded = encoders[column_name].transform(exploded.values.reshape(-1, 1))
    encoded_df = pd.DataFrame(encoded, columns=encoders[column_name].get_feature_names_out([prefix]))
    return encoded_df.groupby(level=0).sum()


def preprocess(df, is_train: bool = True, train_df: pd.DataFrame = None):
    global categories_which_need_medcard, premium_scaler, experience_scaler, salary_scaler, places_scaler
    # Drop unnecessary columns
    df = df.drop('id', axis=1)
    df = df.drop('contact_person', axis=1)
    df = df.drop('deleted', axis=1)
    df = df.drop('metro_ids', axis=1)
    df = df.drop('is_moderated', axis=1)
    df = df.drop('regionNameTerm', axis=1)
    df = df.drop('company_name', axis=1)
    df = df.drop('contact_source', axis=1)
    df = df.drop('data_ids', axis=1)
    df = df.drop('foreign_workers_capability', axis=1)
    df = df.drop('is_uzbekistan_recruitment', axis=1)
    df = df.drop('oknpo_code', axis=1)
    df = df.drop('okso_code', axis=1)
    df = df.drop('publication_period', axis=1)
    df = df.drop('contactList', axis=1)
    df = df.drop('regionName', axis=1)
    df = df.drop('retraining_capability', axis=1)
    df = df.drop('retraining_condition', axis=1)
    df = df.drop('retraining_grant_value', axis=1)
    df = df.drop('vacancy_address_additional_info',axis=1)
    df = df.drop('vacancy_address',axis=1)
    df = df.drop('vacancy_address_code',axis=1)
    df = df.drop('vacancy_address_house',axis=1)
    df = df.drop('vacancy_address_latitude',axis=1)
    df = df.drop('vacancy_address_longitude',axis=1)
    df = df.drop('industryBranchName',axis=1)
    df = df.drop('full_company_name',axis=1)
    df = df.drop('company_inn',axis=1)
    df = df.drop('company',axis=1)
    df = df.drop('company_code',axis=1)
    df = df.drop('code_external_system',axis=1)
    df = df.drop('visibility',axis=1)
    df = df.drop('additional_requirements', axis=1)
    df = df.drop('education_speciality', axis=1)
    df = df.drop('other_vacancy_benefit', axis=1)
    df = df.drop('position_requirements', axis=1)
    df = df.drop('position_responsibilities', axis=1)
    df = df.drop('required_certificates', axis=1)
    df = df.drop('hardSkills', axis=1)
    df = df.drop('softSkills', axis=1)
    df = df.drop('required_drive_license', axis=1)
    df = df.drop('languageKnowledge', axis=1)
    
    ## Trying without date
    df = df.drop('change_time', axis=1)
    df = df.drop('date_create', axis=1)
    df = df.drop('date_modify', axis=1)
    df = df.drop('published_date', axis=1)
    
    
    
    # Deal with NaN
    df['academic_degree'].fillna('absent', inplace=True)
    df['accommodation_type'].fillna('absent', inplace=True)
    #df['additional_requirements'].fillna('absent', inplace=True)
    df['bonus_type'].fillna('absent', inplace=True)
    df['measure_type'].fillna('absent', inplace=True)
    #df['education_speciality'].fillna('absent', inplace=True)
    #df['other_vacancy_benefit'].fillna('absent', inplace=True)
    #df['position_requirements'].fillna('absent', inplace=True)
    #df['position_responsibilities'].fillna('absent', inplace=True)
    #df['required_certificates'].fillna('absent', inplace=True)
    df['social_protected_ids'].fillna('absent', inplace=True)
    df['transport_compensation'].fillna('absent', inplace=True)
    df['vacancy_benefit_ids'].fillna('absent', inplace=True)
    df['professionalSphereName'].fillna('absent', inplace=True)
    df['is_mobility_program'].fillna(False, inplace=True)
    df['additional_premium'].fillna(0, inplace=True)
    df['required_experience'].fillna(0, inplace=True)
    if is_train:
        df = impute_values(df,is_train=True, train_df=df)
    else:
        df = impute_values(df,is_train=False,train_df=train_df)
    categories_which_need_medcard = set(df[df['need_medcard'] == True]['code_professional_sphere'])
    df['need_medcard'] = df['need_medcard'].fillna(df['code_professional_sphere'].apply(needs_medcard))
        
    ### Transform
    df['accommodation_capability'] = df['accommodation_capability'].astype(int)
    df['career_perspective'] = df['career_perspective'].astype(int)
    df['is_mobility_program'] = df['is_mobility_program'].astype(int)
    df['is_quoted'] = df['is_quoted'].astype(int)
    df['need_medcard'] = df['need_medcard'].astype(int)
    df['retraining_grant'] = df['retraining_grant'].replace({'нет стипендии': 0, 'есть стипендия': 1})
    n_features = 100
    if is_train:
        df = df.dropna(subset=['state_region_code'])
        df = df.dropna(subset=['federalDistrictCode'])
        df = df.loc[df['salary'] <= 400000]
        df = pd.concat([df.drop('academic_degree',axis=1),encode_column_train(df, 'academic_degree', 'academic_degree')],axis=1)
        df = pd.concat([df.drop('accommodation_type',axis=1),encode_column_train(df, 'accommodation_type', 'accommodation_type')],axis=1)
        df = pd.concat([df.drop('bonus_type',axis=1),encode_column_train(df, 'bonus_type', 'bonus_type')],axis=1)
        df = pd.concat([df.drop('measure_type',axis=1),encode_column_train(df, 'measure_type', 'measure_type')],axis=1)
        df = pd.concat([df.drop('busy_type',axis=1),encode_column_train(df, 'busy_type', 'busy_type')],axis=1)
        df = pd.concat([df.drop('education',axis=1),encode_column_train(df, 'education', 'education')],axis=1)
        df = pd.concat([df.drop('original_source_type',axis=1),encode_column_train(df, 'original_source_type', 'original_source_type')],axis=1)
        df = pd.concat([df.drop('company_business_size',axis=1),encode_column_train(df, 'company_business_size', 'company_business_size')],axis=1)
        df = pd.concat([df.drop('schedule_type',axis=1),encode_column_train(df, 'schedule_type', 'schedule_type')],axis=1)
        df = pd.concat([df.drop('social_protected_ids',axis=1),encode_column_train(df, 'social_protected_ids', 'social_protected_ids')],axis=1)
        df = pd.concat([df.drop('source_type',axis=1),encode_column_train(df, 'source_type', 'source_type')],axis=1)
        df = pd.concat([df.drop('state_region_code',axis=1),encode_column_train(df, 'state_region_code', 'state_region_code')],axis=1)
        df = pd.concat([df.drop('status',axis=1),encode_column_train(df, 'status', 'status')],axis=1)
        df = pd.concat([df.drop('transport_compensation',axis=1),encode_column_train(df, 'transport_compensation', 'transport_compensation')],axis=1)
        df = pd.concat([df.drop('vacancy_benefit_ids',axis=1),encode_column_train(df, 'vacancy_benefit_ids', 'vacancy_benefit_ids')],axis=1)
        df = pd.concat([df.drop('professionalSphereName',axis=1),encode_column_train(df, 'professionalSphereName', 'professionalSphereName')],axis=1)
        df = pd.concat([df.drop('federalDistrictCode',axis=1),encode_column_train(df, 'federalDistrictCode', 'federalDistrictCode')],axis=1)
        df = pd.concat([df.drop('code_professional_sphere',axis=1),encode_column_train(df, 'code_professional_sphere', 'code_professional_sphere')],axis=1)
        
        
        premium_scaler = StandardScaler()
        df['additional_premium'] = premium_scaler.fit_transform(np.array(df['additional_premium']).reshape(-1,1))
        experience_scaler = StandardScaler()
        df['required_experience'] = experience_scaler.fit_transform(np.array(df['required_experience']).reshape(-1,1))
        
        mean_salary_max = df.loc[df['salary_max'] > 0, 'salary_max'].mean()
        df['salary_max'] = df['salary_max'].replace(0, mean_salary_max)
        df.loc[df['salary'] == 0, 'salary'] = df['salary_max']
        salary_scaler = StandardScaler()
        df['salary'] = salary_scaler.fit_transform(np.array(df['salary']).reshape(-1,1))
        places_scaler = StandardScaler()
        df['work_places'] = places_scaler.fit_transform(np.array(df['work_places']).reshape(-1,1))
        df = hash_col(df, 'code_profession',100)
        df = df.dropna()
    else:
        df = df.fillna(train_df.median())
        df = pd.concat([df.drop('academic_degree',axis=1),encode_column(df, 'academic_degree', 'academic_degree')],axis=1)
        df = pd.concat([df.drop('accommodation_type',axis=1),encode_column(df, 'accommodation_type', 'accommodation_type')],axis=1)
        df = pd.concat([df.drop('bonus_type',axis=1),encode_column(df, 'bonus_type', 'bonus_type')],axis=1)
        df = pd.concat([df.drop('measure_type',axis=1),encode_column(df, 'measure_type', 'measure_type')],axis=1)
        df = pd.concat([df.drop('busy_type',axis=1),encode_column(df, 'busy_type', 'busy_type')],axis=1)
        df = pd.concat([df.drop('education',axis=1),encode_column(df, 'education', 'education')],axis=1)
        df = pd.concat([df.drop('original_source_type',axis=1),encode_column(df, 'original_source_type', 'original_source_type')],axis=1)
        df = pd.concat([df.drop('company_business_size',axis=1),encode_column(df, 'company_business_size', 'company_business_size')],axis=1)
        df = pd.concat([df.drop('schedule_type',axis=1),encode_column(df, 'schedule_type', 'schedule_type')],axis=1)
        df = pd.concat([df.drop('social_protected_ids',axis=1),encode_column(df, 'social_protected_ids', 'social_protected_ids')],axis=1)
        df = pd.concat([df.drop('source_type',axis=1),encode_column(df, 'source_type', 'source_type')],axis=1)
        df = pd.concat([df.drop('state_region_code',axis=1),encode_column(df, 'state_region_code', 'state_region_code')],axis=1)
        df = pd.concat([df.drop('status',axis=1),encode_column(df, 'status', 'status')],axis=1)
        df = pd.concat([df.drop('transport_compensation',axis=1),encode_column(df, 'transport_compensation', 'transport_compensation')],axis=1)
        df = pd.concat([df.drop('vacancy_benefit_ids',axis=1),encode_column(df, 'vacancy_benefit_ids', 'vacancy_benefit_ids')],axis=1)
        df = pd.concat([df.drop('professionalSphereName',axis=1),encode_column(df, 'professionalSphereName', 'professionalSphereName')],axis=1)
        df = pd.concat([df.drop('federalDistrictCode',axis=1),encode_column(df, 'federalDistrictCode', 'federalDistrictCode')],axis=1)
        df = pd.concat([df.drop('code_professional_sphere',axis=1),encode_column(df, 'code_professional_sphere', 'code_professional_sphere')],axis=1)
        df['additional_premium'] = premium_scaler.transform(np.array(df['additional_premium']).reshape(-1,1))
        df['required_experience'] = experience_scaler.transform(np.array(df['required_experience']).reshape(-1,1))
        df['work_places'] = places_scaler.transform(np.array(df['work_places']).reshape(-1,1))
        df = hash_col(df, 'code_profession',100)
        df = df.fillna(0)
    df = df.drop('vacancy_name', axis=1)
    if is_train or 'salary_min' in df.columns:
        df = df.drop('salary_min', axis=1)
        df = df.drop('salary_max', axis=1)

    return df

print('train')
df = preprocess(df)

df.head()

train


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,accommodation_capability,additional_premium,career_perspective,is_mobility_program,is_quoted,need_medcard,required_experience,retraining_grant,salary,work_places,academic_degree_absent,academic_degree_Доктор наук,academic_degree_Кандидат наук,accommodation_type_DORMITORY,accommodation_type_FLAT,accommodation_type_HOUSE,accommodation_type_ROOM,accommodation_type_absent,bonus_type_ANNUAL,bonus_type_MONTHLY,bonus_type_QUARTERLY,bonus_type_absent,measure_type_PERCENT,measure_type_RUBLE,measure_type_absent,busy_type_Временная,busy_type_Полная занятость,busy_type_Сезонная,busy_type_Стажировка,busy_type_Удаленная,busy_type_Частичная занятость,education_Высшее,education_Не указано,education_Незаконченное высшее,education_Основное общее,education_Среднее,education_Среднее общее,education_Среднее профессиональное,original_source_type_CHAZ,original_source_type_COMPANY,...,code_profession_60,code_profession_61,code_profession_62,code_profession_63,code_profession_64,code_profession_65,code_profession_66,code_profession_67,code_profession_68,code_profession_69,code_profession_70,code_profession_71,code_profession_72,code_profession_73,code_profession_74,code_profession_75,code_profession_76,code_profession_77,code_profession_78,code_profession_79,code_profession_80,code_profession_81,code_profession_82,code_profession_83,code_profession_84,code_profession_85,code_profession_86,code_profession_87,code_profession_88,code_profession_89,code_profession_90,code_profession_91,code_profession_92,code_profession_93,code_profession_94,code_profession_95,code_profession_96,code_profession_97,code_profession_98,code_profession_99
0,0.0,-0.078048,0.0,0.0,0.0,1.0,-0.533655,0.0,-0.260676,-0.104307,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,-0.078048,0.0,0.0,0.0,1.0,-0.533655,0.0,0.856707,-0.104307,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-0.078048,0.0,0.0,0.0,1.0,-0.533655,0.0,-0.619082,-0.001685,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-0.078048,0.0,0.0,0.0,1.0,-0.533655,0.0,-0.861870,-0.104307,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,-0.078048,0.0,0.0,0.0,1.0,-0.533655,0.0,-0.536522,-0.104307,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor, Pool


X_train = df.drop('salary',axis=1)
y_train = df['salary']

model = CatBoostRegressor(
    iterations=1000,
    depth=16,
    learning_rate=0.15,
    loss_function='RMSE',
    gpu_ram_part=0.8,
    task_type='GPU',
    snapshot_interval=1000,
)
print('start training')
model.fit(X_train, y_train)
print('trained')


start training
0:	learn: 0.9413006	total: 424ms	remaining: 7m 3s
1:	learn: 0.9021359	total: 562ms	remaining: 4m 40s
2:	learn: 0.8717031	total: 699ms	remaining: 3m 52s
3:	learn: 0.8474252	total: 834ms	remaining: 3m 27s
4:	learn: 0.8286156	total: 972ms	remaining: 3m 13s
5:	learn: 0.8132118	total: 1.11s	remaining: 3m 3s
6:	learn: 0.8006436	total: 1.24s	remaining: 2m 56s
7:	learn: 0.7889548	total: 1.38s	remaining: 2m 50s
8:	learn: 0.7793082	total: 1.51s	remaining: 2m 46s
9:	learn: 0.7711610	total: 1.65s	remaining: 2m 43s
10:	learn: 0.7640832	total: 1.79s	remaining: 2m 40s
11:	learn: 0.7582138	total: 1.92s	remaining: 2m 38s
12:	learn: 0.7517650	total: 2.06s	remaining: 2m 36s
13:	learn: 0.7458725	total: 2.19s	remaining: 2m 34s
14:	learn: 0.7408493	total: 2.33s	remaining: 2m 32s
15:	learn: 0.7359047	total: 2.46s	remaining: 2m 31s
16:	learn: 0.7326324	total: 2.6s	remaining: 2m 30s
17:	learn: 0.7295180	total: 2.74s	remaining: 2m 29s
18:	learn: 0.7264081	total: 2.87s	remaining: 2m 28s
19:	learn:

In [13]:
import joblib
model.save_model('regression.cbm')
for key, encoder in encoders.items():
    filename = f"{key}_encoder.joblib"
    joblib.dump(encoder, filename)
    print(f"Saved encoder to {filename}")
joblib.dump(premium_scaler,'premium_scaler.joblib')
joblib.dump(experience_scaler,'experience_scaler.joblib')
joblib.dump(salary_scaler,'salary_scaler.joblib')
joblib.dump(places_scaler,'places_scaler.joblib')

Saved encoder to academic_degree_encoder.joblib
Saved encoder to accommodation_type_encoder.joblib
Saved encoder to bonus_type_encoder.joblib
Saved encoder to measure_type_encoder.joblib
Saved encoder to busy_type_encoder.joblib
Saved encoder to education_encoder.joblib
Saved encoder to original_source_type_encoder.joblib
Saved encoder to company_business_size_encoder.joblib
Saved encoder to schedule_type_encoder.joblib
Saved encoder to social_protected_ids_encoder.joblib
Saved encoder to source_type_encoder.joblib
Saved encoder to state_region_code_encoder.joblib
Saved encoder to status_encoder.joblib
Saved encoder to transport_compensation_encoder.joblib
Saved encoder to vacancy_benefit_ids_encoder.joblib
Saved encoder to professionalSphereName_encoder.joblib
Saved encoder to federalDistrictCode_encoder.joblib
Saved encoder to code_professional_sphere_encoder.joblib
Saved encoder to required_drive_license_encoder.joblib
Saved encoder to languageKnowledge_encoder.joblib


['places_scaler.joblib']